# File Case
- This is used to cache the Dataframe result, even there are multiply Dataframe, which can help to reduce the huge time in feature engineering
- It also support to log the function time cost and parameters


## Installation
pip install file_cache

## Sample case

In [1]:
from  file_cache.cache import file_cache
import numpy  as np
import pandas as pd

@file_cache()
def test_cache_normal(name):
    import time
    import numpy  as np
    time.sleep(3)
    return pd.DataFrame(data= np.arange(0,10).reshape(2,5))

normal_df = test_cache_normal('Felix')
normal_df.head()

2018-12-25 22:40:42,026 util_log.py[61] DEBUG Start the program at:LALI2-M-G0MD, 127.0.0.1, with:Load module
2018-12-25 22:40:42,029 util_log.py[41] INFO Begin:test_cache_normal(1 paras) with:['Felix'], []
2018-12-25 22:40:42,178 cache.py[29] DEBUG try to read cache from file:./cache/test_cache_normal=Felix=.h5, (h5, key:['/df_0'])
2018-12-25 22:40:42,189 util_log.py[49] INFO Cost:   0.16 sec:'test_cache_normal'(1 paras)(['Felix'], []), return:DataFrame, end 


,0,1,2,3,4
0,0,1,2,3,4
1,5,6,7,8,9


## Return mulpiple DF with tuple
Support to cache multiple DF with tuple

In [2]:
import time
from functools import lru_cache

@lru_cache()
@file_cache()
def test_cache_tuple(name):
    time.sleep(3)
    df0 = pd.DataFrame(data= np.arange(5,15).reshape(2,5))
    df1 = pd.DataFrame(data= np.arange(20,30).reshape(2,5))
    return df0, df1

df0, df1 = test_cache_tuple('Felix2')
print(df0 , '\n')
print(df1)

2018-12-25 22:40:42,227 util_log.py[41] INFO Begin:test_cache_tuple(1 paras) with:['Felix2'], []
2018-12-25 22:40:42,252 cache.py[29] DEBUG try to read cache from file:./cache/test_cache_tuple=Felix2=.h5, (h5, key:['/df_0', '/df_1'])
2018-12-25 22:40:42,273 util_log.py[49] INFO Cost:   0.05 sec:'test_cache_tuple'(1 paras)(['Felix2'], []), return:tuple, end 


    0   1   2   3   4
0   5   6   7   8   9
1  10  11  12  13  14 

    0   1   2   3   4
0  20  21  22  23  24
1  25  26  27  28  29


## For the input paras can not be cached
If the input is DF or cannot be hashed, ignore the cache, run the function directly

In [3]:
@file_cache()
def test_cache_ignore(name):
    df0 = pd.DataFrame(data= np.arange(5,15).reshape(2,5))
    return df0

df = pd.DataFrame(data= np.arange(5,15).reshape(2,5))
ignore = test_cache_ignore(df)


2018-12-25 22:40:42,302 util_log.py[41] INFO Begin:test_cache_ignore(1 paras) with:['DataFrame'], []
2018-12-25 22:40:42,307 cache.py[112] DEBUG There is DataFrame in the args
2018-12-25 22:40:42,309 cache.py[94] WARNING Don not support cache for fn:test_cache_ignore, para:DataFrame, kw:{}
2018-12-25 22:40:42,312 util_log.py[49] INFO Cost:   0.01 sec:'test_cache_ignore'(1 paras)(['DataFrame'], []), return:DataFrame, end 


## Log the function time and parameter

In [1]:
from file_cache.utils.util_log import *
@timed()
def log_time(arg):
    return f'{arg} msg'

print(log_time("hello"))

2018-12-26 11:08:52,662 util_log.py[61] DEBUG Start the program at:LALI2-M-G0MD, 127.0.0.1, with:Load module
2018-12-26 11:08:52,665 util_log.py[41] INFO log_time begin with(1 paras) :['hello'], []
2018-12-26 11:08:52,667 util_log.py[49] INFO log_time cost:   0.00 sec:(1 paras)(['hello'], []), return:hello msg, end 


hello msg


## Not only support DataFrame, but also support Series

In [4]:
from  file_cache.cache import file_cache
@file_cache()
def get_train_data():
    from sklearn import datasets
    import pandas as pd
    import numpy as np
    data = datasets.load_boston()
    df = pd.DataFrame( data.data , columns=data.feature_names)
    df['target'] = data.target
    df.head()
    return df, df['target']

df, series = get_train_data()
print(type(df), type(series))

df, series = get_train_data()
print(type(df), type(series))


2018-12-26 11:16:24,937 util_log.py[41] INFO get_train_data begin with(0 paras) :[], []
2018-12-26 11:16:24,941 cache.py[38] DEBUG Can not find cache from file:./cache/get_train_data==.h5
2018-12-26 11:16:24,949 cache.py[62] DEBUG ====Write 506 records to File#./cache/get_train_data==.h5, with:df_0
2018-12-26 11:16:25,142 cache.py[62] DEBUG ====Write 506 records to File#./cache/get_train_data==.h5, with:df_1
2018-12-26 11:16:25,151 util_log.py[49] INFO get_train_data cost:   0.21 sec:(0 paras)([], []), return:tuple, end 
2018-12-26 11:16:25,156 util_log.py[41] INFO get_train_data begin with(0 paras) :[], []
2018-12-26 11:16:25,170 cache.py[29] DEBUG try to read cache from file:./cache/get_train_data==.h5, (h5, key:['/df_0', '/df_1'])
2018-12-26 11:16:25,186 util_log.py[49] INFO get_train_data cost:   0.03 sec:(0 paras)([], []), return:tuple, end 


<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
